In [2]:
import torch

model = torch.load('logs/lightning_logs/version_2/checkpoints/epoch=00-valid/AUROC=1.0000.ckpt')

In [3]:
for key, value in model.items():
    if key == 'epoch' or key == 'global_step':
        print(key, value)

epoch 0
global_step 310


In [4]:
state_dict = model['state_dict']
from collections import OrderedDict
new_state_dict = OrderedDict()
for key, value in state_dict.items():
    key = key.replace('model.', '')
    new_state_dict[key] = value
for key, value in new_state_dict.items():
    print(key)

encoder.conv1.weight
encoder.bn1.weight
encoder.bn1.bias
encoder.bn1.running_mean
encoder.bn1.running_var
encoder.bn1.num_batches_tracked
encoder.conv2.weight
encoder.bn2.weight
encoder.bn2.bias
encoder.bn2.running_mean
encoder.bn2.running_var
encoder.bn2.num_batches_tracked
encoder.block1.skip.weight
encoder.block1.skipbn.weight
encoder.block1.skipbn.bias
encoder.block1.skipbn.running_mean
encoder.block1.skipbn.running_var
encoder.block1.skipbn.num_batches_tracked
encoder.block1.rep.0.conv1.weight
encoder.block1.rep.0.pointwise.weight
encoder.block1.rep.1.weight
encoder.block1.rep.1.bias
encoder.block1.rep.1.running_mean
encoder.block1.rep.1.running_var
encoder.block1.rep.1.num_batches_tracked
encoder.block1.rep.3.conv1.weight
encoder.block1.rep.3.pointwise.weight
encoder.block1.rep.4.weight
encoder.block1.rep.4.bias
encoder.block1.rep.4.running_mean
encoder.block1.rep.4.running_var
encoder.block1.rep.4.num_batches_tracked
encoder.block2.skip.weight
encoder.block2.skipbn.weight
encode

In [5]:
from model.network.Recce import Recce

model = Recce(1)
model.load_state_dict(new_state_dict, strict=True)

/opt/conda/envs/deepfake_detector/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/workspace/minsung/Personal_DeepFake_Detect/deepfake_detector_ddp/model/network/Recce.py:25: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  self.encoder = encoder_params[self.name]["init_op"]()


<All keys matched successfully>

In [7]:
from torch.utils import data
from dataset.custom_deepfake import Custom_Dataset
import os.path as osp

class Custom_Loader():
    def __init__(self, root_path, data_cfg):
        super().__init__()

        self.data_cfg = data_cfg
        self.train_dataset = Custom_Dataset(root_path=osp.join(root_path, 'train'))
        self.valid_dataset = Custom_Dataset(root_path=osp.join(root_path, 'val'))
        self.test_dataset = Custom_Dataset(root_path=osp.join(root_path, 'test'))
        self.prepare_data_per_node = True
    
    # def prepare_data(self):
    #     pass # TODO

    # def setup(self, stage=None):
    #     pass # TODO
        
    def train_dataloader(self):
        self.train_loader = data.DataLoader(self.train_dataset, shuffle=True,
                                            num_workers=self.data_cfg.get("num_workers", 8),
                                            batch_size=self.data_cfg["train_batch_size"],
                                            pin_memory=True,
                                            drop_last=True)
        return self.train_loader
    
    def val_dataloader(self):
        self.val_loader = data.DataLoader(self.valid_dataset, shuffle=False,
                                        num_workers=self.data_cfg.get("num_workers", 8),
                                        batch_size=self.data_cfg["val_batch_size"],
                                        pin_memory=True)
        return self.val_loader

    def test_dataloader(self):
        self.test_loader = data.DataLoader(self.test_dataset, shuffle=False,
                                        num_workers=self.data_cfg.get("num_workers", 8),
                                        batch_size=self.data_cfg["val_batch_size"],
                                        pin_memory=True)
        return self.test_loader

data_cfg = dict()
data_cfg['train_batch_size'] = 32
data_cfg['val_batch_size'] = 32
data_cfg['test_batch_size'] = 32
loader = Custom_Loader(root_path='../data/deepfake_classification', data_cfg=data_cfg)
train_loader = loader.train_dataloader()
val_loader = loader.val_dataloader()
test_loader = loader.test_dataloader()
print(len(train_loader.dataset))
print(len(val_loader.dataset))
print(len(test_loader.dataset))

69605
19887
9944


In [22]:
model.eval()
model = model.cuda()
total_preds = list()
total_labels = list()
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.cuda()
        labels = labels.cuda()
        out = model(imgs)
        total_preds.extend(torch.sigmoid(out).cpu().numpy())
        total_labels.extend(labels.cpu().numpy())

In [33]:
print(len(total_preds))
print(len(total_labels))

9944
9944


In [34]:
from sklearn.metrics import roc_auc_score

auroc = roc_auc_score(total_labels, total_preds)
print(auroc)

1.0


In [31]:
one = sum(total_labels)
print(one)
print(len(total_labels) - one)

2494
7450


In [49]:
print(sum(total_preds))
print(sum(total_labels))
for i in range(len(total_preds)):
    if total_preds[i] > total_labels[i]:
        print(total_preds[i], total_labels[i])

[2810.9004]
2494
[0.04452275] 0
[0.06789428] 0
[0.05272238] 0
[0.05915622] 0
[0.05344684] 0
[0.04181082] 0
[0.06624015] 0
[0.03851932] 0
[0.04150694] 0
[0.03699662] 0
[0.04718331] 0
[0.05310347] 0
[0.05968197] 0
[0.04735822] 0
[0.05521425] 0
[0.0327928] 0
[0.04764217] 0
[0.05583446] 0
[0.03469997] 0
[0.04502438] 0
[0.04355523] 0
[0.05405641] 0
[0.04457032] 0
[0.09318927] 0
[0.03400763] 0
[0.05376413] 0
[0.04870837] 0
[0.04005358] 0
[0.05437674] 0
[0.04307094] 0
[0.05038011] 0
[0.05332486] 0
[0.04779552] 0
[0.03973555] 0
[0.04227719] 0
[0.06238526] 0
[0.04860565] 0
[0.04665753] 0
[0.05074627] 0
[0.07420027] 0
[0.03665482] 0
[0.04264619] 0
[0.05311924] 0
[0.04325477] 0
[0.04938986] 0
[0.03898352] 0
[0.06234618] 0
[0.06740125] 0
[0.05009855] 0
[0.05568735] 0
[0.04171969] 0
[0.04177656] 0
[0.06016859] 0
[0.05078929] 0
[0.05854553] 0
[0.04974524] 0
[0.04635815] 0
[0.05258175] 0
[0.05405305] 0
[0.04644974] 0
[0.04487073] 0
[0.05510155] 0
[0.05114317] 0
[0.03947438] 0
[0.09212032] 0
[0.072972

In [48]:
preds = (torch.tensor(total_preds) >= 0.5).int().squeeze(-1)
labels = torch.tensor(total_labels)
print(preds.shape)
print(labels.shape)
print(preds.sum())
print(labels.sum())
print(torch.sum(preds==labels))


torch.Size([9944])
torch.Size([9944])
tensor(2494)
tensor(2494)
tensor(9944)
